In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import log, dot
import pandas as pd

%load_ext lab_black
%load_ext watermark

# Arrhythmia

A logistic regression example.

Adapted from [unit 7: arrhythmia.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/arrhythmia.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/arrhythmia.csv).

Associated lecture video: Unit 7 Lesson 15

## Problem statement

Patients who undergo Coronary Artery Bypass Graft Surgery (CABG) have an approximate 19-40% chance of developing atrial fibrillation (AF). AF can lead to blood clots forming causing greater in-hospital mortality, strokes, and longer hospital stays. While this can be prevented with drugs, it is very expensive and sometimes dangerous if not warranted. Ideally, several risk factors which would indicate an increased risk of developing AF in this population could save lives and money by indicating which patients need pharmacological intervention. Researchers began collecting data from CABG patients during their hospital stay such as demographics like age and sex, as well as heart rate, cholesterol, operation time, etc.. Then, the researchers recorded which patients developed AF during their hospital stay. Researchers now want to find those pieces of data which indicate high risk of AF. In the past, indicators like age, hypertension, and body surface area (BSA) have been good indicators, though these alone have not produced a satisfactory solution.

Fibrillation occurs when the heart muscle begins a quivering motion instead of a normal, healthy pumping rhythm. Fibrillation can affect the atrium (atrial fibrillation) or the ventricle (ventricular  fibrillation); ventricular fibrillation is imminently life threatening.

Atrial fibrillation is the quivering, chaotic motion in the upper chambers of the heart, known as the atria. Atrial fibrillation is often due to serious underlying medical conditions, and should be evaluated by a physician. It is not typically a medical emergency.

Ventricular fibrillation occurs in the ventricles (lower chambers) of the heart; it is always a medical emergency. If left untreated, ventricular fibrillation (VF, or V-fib) can lead to death within minutes. When a heart goes into V-fib, effective pumping of the blood stops. V-fib is considered a form of cardiac arrest, and an individual suffering from it will not survive unless cardiopulmonary resuscitation (CPR) and defibrillation are provided immediately.

DATA Arrhythmia
- Y = Fibrillation
- X1 = Age
- X2 = Aortic Cross Clamp Time
- X3 = Cardiopulmonary Bypass Time:
    - Bypass of the heart and lungs as, for example, in open heart surgery. Blood returning to the heart is diverted through a heart-lung machine (a pump-oxygenator) before returning it to the arterial circulation. The machine does the work both of the heart (pump blood) and the lungs (supply oxygen to red blood cells).
- X4 = ICU Time	(Intensive Care Unit)
- X5 = Avg Heart Rate	
- X6 = Left Ventricle Ejection Fraction
- X7 = Hypertension
- X8 = Gender [1 -Female; 0-Male]
- X9 = Diabetis
- X10 = Previous MI

In [3]:
data_df = pd.read_csv("../data/arrhythmia.csv")
data_df.info()
X = data_df.iloc[:, 1:].to_numpy()
# add intercept column to X
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = data_df["Fibrillation"].to_numpy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Fibrillation                   81 non-null     float64
 1   Age                            81 non-null     float64
 2   AorticCrossClampTime           81 non-null     float64
 3   CardiopulmonaryBypassTime      81 non-null     float64
 4   ICUTime                        81 non-null     float64
 5   AvgHeartRate                   81 non-null     float64
 6   LeftVentricleEjectionFraction  81 non-null     float64
 7   Hypertension                   81 non-null     float64
 8   Gender                         81 non-null     float64
 9   Diabetes                       81 non-null     float64
 10  PreviousMI                     81 non-null     float64
dtypes: float64(11)
memory usage: 7.1 KB


In [4]:
with pm.Model() as m:
    X_data = pm.Data("X_data", X_aug)
    y_data = pm.Data("y_data", y)

    betas = pm.Normal("beta", mu=0, tau=0.001, shape=X.shape[1] + 1)

    p = dot(X_data, betas)

    lik = pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(
        10000,
        chains=4,
        tune=500,
        cores=4,
        random_seed=1,
    )

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/data.py:667: FutureWarning: The `mutable` kwarg was not specified. Currently it defaults to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default will change to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Set `pm.Data(..., mutable=False/True)`, or use `pm.ConstantData`/`pm.MutableData`.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta]


Sampling 4 chains for 500 tune and 10_000 draw iterations (2_000 + 40_000 draws total) took 92 seconds.


In [5]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-13.023,4.894,-22.508,-3.429,0.038,0.027,16609.0,21883.0,1.0
beta[1],0.188,0.050,0.095,0.288,0.000,0.000,24380.0,24247.0,1.0
beta[2],0.033,0.025,-0.014,0.086,0.000,0.000,17851.0,22892.0,1.0
beta[3],-0.023,0.016,-0.056,0.007,0.000,0.000,18807.0,23149.0,1.0
beta[4],-0.154,0.096,-0.344,0.032,0.001,0.000,33092.0,26041.0,1.0
beta[5],0.006,0.031,-0.054,0.069,0.000,0.000,24147.0,26118.0,1.0
beta[6],0.024,0.028,-0.031,0.080,0.000,0.000,25153.0,26291.0,1.0
beta[7],-0.655,0.675,-1.970,0.683,0.004,0.003,33643.0,26703.0,1.0
beta[8],-0.330,0.688,-1.671,1.033,0.003,0.003,39163.0,28282.0,1.0
beta[9],1.308,0.691,-0.016,2.698,0.004,0.003,26892.0,24877.0,1.0


In [6]:
%watermark -n -u -v -iv -p aesara,aeppl

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

numpy : 1.22.4
pandas: 1.4.2
arviz : 0.12.1
pymc  : 4.0.0

